In [ ]:
import junodch_utils_read_img as utils

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.keras import TqdmCallback

import keras
from keras import layers, losses
from keras.models import Model
from shapely.geometry import box
#import gdal

import rasterio
from rasterio import plot as rastPlt
from rasterio.merge import merge as rasterMerge


# Data preparation
### Fetch data from file

In [ ]:
folderName = "img/Sokoto/"
satelliteFile = "Sentinel-2.tif"
#nightFile = "Night VIIRS_1.tif"
nightFile = "lowres_night_1.tif"

# Filter the night tile not covering the day area.
with rasterio.open(folderName + satelliteFile) as s:
  sBox = box(*s.bounds).exterior.coords
aoi = []
for v in sBox:
  aoi.append((v[0], v[1]))
aoi.pop()

train, test = utils.getTrainingAndTestPerimeter(folderName + nightFile, 200, area=aoi)

print('TrainingTile:',len(train))
print('TestTile:',len(test))

with rasterio.open(folderName + satelliteFile) as s:
  data, meta = utils.getEachImgFromCoord(s, train, True)

### Visualize training tile

In [ ]:
utils.displayTiles(data, meta)

### Format data for autoencoder

In [ ]:
print(data[0].shape)
#print(meta[0])

dataTrain_formated = utils.formatDataForAutoencoder(data, res=64)
print(dataTrain_formated.shape)

# Autoencoder

In [ ]:
# Input encoder
input_img = keras.Input(shape=(64,64,4))

#optimizer = 'adam'
optimizer = keras.optimizers.Adam(
  learning_rate=0.001,
  #learning_rate=0.002,
  beta_1=0.9,
  beta_2=0.999,
)
#lossFunction = keras.losses.MeanAbsoluteError() # L1
lossFunction = keras.losses.MeanSquaredError() # l2
#lossFunction = tf.keras.losses.KLDivergence() # give blanc result ?
activationFunction = lambda x: tf.keras.activations.relu(x, max_value=255)
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=5, patience=5)

# (5,5) good ?
# 20,8,8,8,8,8
cnn = layers.Conv2D(20,(5,5), padding='same', activation=activationFunction)(input_img)
cnn = layers.MaxPool2D((2,2), padding='same')(cnn)
cnn = layers.Conv2D(8,(3,3), padding='same', activation=activationFunction)(cnn)
cnn = layers.MaxPool2D((2,2), padding='same')(cnn)
cnn = layers.Conv2D(8,(3,3), padding='same', activation=activationFunction)(cnn)
encoded = layers.MaxPool2D((2,2), padding='same')(cnn)
print('Encoder shape:',encoded.get_shape())
encoder = keras.Model(input_img, encoded)

cnn = layers.Conv2D(8,(3,3), padding='same', activation=activationFunction)(encoded)
cnn = layers.UpSampling2D((2,2))(cnn)
cnn = layers.Conv2D(8,(3,3), padding='same', activation=activationFunction)(cnn)
cnn = layers.UpSampling2D((2,2))(cnn)
cnn = layers.Conv2D(8,(3,3), padding='same', activation=activationFunction)(cnn)
cnn = layers.UpSampling2D((2,2))(cnn)
decoder = layers.Conv2D(4, (3,3), padding='same', activation=activationFunction)(cnn)

autoencoder = keras.Model(input_img, decoder)
autoencoder.compile(optimizer=optimizer, loss=lossFunction)

result = autoencoder.fit(dataTrain_formated[0:600], dataTrain_formated[0:600],
                          epochs=100,
                          #batch_size=100,
                          #shuffle=True,
                          verbose=0,
                          #validation_data=(dataTrain_formated[0:2000], dataTrain_formated[0:2000]),
                          callbacks=[
                            TqdmCallback(verbose=1),
                            earlyStop,
                          ],
                        )


In [ ]:
plt.plot(result.history['loss'][10:], label='Training')
#autoencoder.summary()
#plt.plot(result.history['val_loss'], label='Testing')

In [ ]:
dataInput = dataTrain_formated[len(dataTrain_formated)-100:]

utils.displayAutoEncoderResults(dataInput, None, autoencoder, lossFunction)

In [ ]:
with rasterio.open(folderName + satelliteFile) as s:
  validation, metaValid = utils.getEachImgFromCoord(s, test[0:1] + test[1104:1123]+ test[2944:2964]+ test[4000:4020]+ test[5000:5020]+ test[10000:10020], True)

utils.displayAutoEncoderResults(utils.formatDataForAutoencoder(validation,res=64), None, autoencoder, lossFunction)

In [ ]:
autoencoder.save('model/autoencoder_64_V2')

In [ ]:
# Display area test
with rasterio.open(folderName + satelliteFile) as s:
  validationTest, metaValidTest = utils.getEachImgFromCoord(s, test[0:10] + test[368:378] + test[736:746] + test[1104:1114] + test[1472:1482], True)

utils.displayTiles(validationTest, metaValidTest)